<a href="https://colab.research.google.com/github/gachet/ad-1-24/blob/main/pca/ejem-pres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # importando numpy
from scipy import stats # importando scipy.stats
import pandas as pd # importando pandas



In [10]:
x = [19,39,30,30,15,15,15,30]
y= [63,74,87,23,35,43,32,73]
datos = np.stack((x, y), axis=0)
datos

array([[19, 39, 30, 30, 15, 15, 15, 30],
       [63, 74, 87, 23, 35, 43, 32, 73]])

In [13]:
A=np.cov(datos)

In [14]:
autovalor, autovector = np.linalg.eig(A)

In [15]:
autovalor

array([ 56.10230167, 580.80841262])

In [16]:
autovector

array([[-0.97124992, -0.23806218],
       [ 0.23806218, -0.97124992]])